### 데이터 불러오기

In [9]:
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from keras.layers import Dropout
from imblearn.over_sampling import SMOTE
import keras.utils
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer

In [31]:
# 새로운 데이터 (EDA)과정 진행 후 데이터
X_train = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/X_fi_train.csv')
X_test = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/X_fi_test.csv')
X_val = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/X_fi_val.csv')
y_train = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/y_fi_train.csv')
y_test = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/y_fi_test.csv')
y_val = pd.read_csv('csv/Multi_to_share/Multi_to_share/fi로 drop 진행/y_fi_val.csv')

In [32]:
# 데이터 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [33]:
y_train = y_train['class']
y_test = y_test['class']
y_val = y_val['class']

In [34]:
# 타겟 맵핑				
label_map = {'Pastry': 0, 'Z_Scratch': 1, 'K_Scatch': 2, 'Stains': 3, 'Dirtiness': 4, 'Bumps': 5, 'Other_Faults': 6}
y_train = [label_map[label] for label in y_train]
y_test = [label_map[label] for label in y_test]
y_val = [label_map[label] for label in y_val]

In [35]:
# y_train을 one-hot encoding으로 변환합니다
num_classes = len(set(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

### 모델 구현 
- 임시로 구현

In [36]:
# 인터넷 참고
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# 모델구성
model = Sequential()
model.add(Dense(input_dim, input_dim=input_dim, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='relu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='relu'))
model.add(Dense(input_dim*2, activation='relu'))
model.add(Dense(input_dim*2, activation='relu'))
model.add(Dense(input_dim//2, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer=custom_opt(0.005), metrics=['accuracy'])

early_stopping = EarlyStopping(patience=20, monitor='val_accuracy')

model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)

print("validation loss, validation accuracy")
loss, acc = model.evaluate(X_val, y_val, verbose = 2)

           

validation loss, validation accuracy
10/10 - 0s - loss: 0.6890 - accuracy: 0.7588 - 26ms/epoch - 3ms/step


In [37]:
# 복습한 내용으로 추가적인 진행
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}
batch_lst = [16, 32, 64, 128, 256]

# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=input_dim))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(64, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(output_dim, activation='softmax'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)
            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'batch': batch}


            
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print(best_time)
print(time_hyper)


10/10 - 0s - loss: 0.2418 - accuracy: 0.7492 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2052 - accuracy: 0.7331 - 27ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2492 - accuracy: 0.7428 - 27ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2116 - accuracy: 0.7492 - 27ms/epoch - 3ms/step
10/10 - 0s - loss: 0.3238 - accuracy: 0.7042 - 27ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2122 - accuracy: 0.7588 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2009 - accuracy: 0.7428 - 19ms/epoch - 2ms/step
10/10 - 0s - loss: 0.2137 - accuracy: 0.6624 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2371 - accuracy: 0.6527 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2003 - accuracy: 0.7138 - 28ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2391 - accuracy: 0.7524 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1876 - accuracy: 0.7524 - 30ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2287 - accuracy: 0.7331 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2218 - accuracy: 0.7653 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1961 - accura

In [38]:
# 복습한 내용으로 추가적인 진행 - 다이아몬드형
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}
batch_lst = [16, 32, 64, 128, 256]

# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=input_dim))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(512, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가   
            model.add(Dense(32, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(32, activation=func))   
            model.add(Dense(output_dim, activation='softmax'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)
            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'batch': batch}


            
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print(best_time)
print(time_hyper)


10/10 - 0s - loss: 0.2027 - accuracy: 0.7331 - 29ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1845 - accuracy: 0.7685 - 30ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2533 - accuracy: 0.7556 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2503 - accuracy: 0.7267 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2424 - accuracy: 0.7138 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1969 - accuracy: 0.7428 - 29ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1925 - accuracy: 0.7492 - 29ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2099 - accuracy: 0.7331 - 29ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2149 - accuracy: 0.7395 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2171 - accuracy: 0.7492 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1921 - accuracy: 0.7492 - 29ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1805 - accuracy: 0.7621 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2117 - accuracy: 0.7588 - 30ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2463 - accuracy: 0.7299 - 29ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2033 - accura

In [39]:
# 복습한 내용으로 추가적인 진행 - 다이아몬드형
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}
batch_lst = [16, 32, 64, 128, 256]

# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            model = Sequential()
            model.add(Dense(512, activation=func, input_dim=input_dim))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(256, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가   
            model.add(Dense(96, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(64, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(32, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(16, activation=func))
            model.add(Dense(12, activation=func)) 
            model.add(Dense(output_dim, activation='softmax'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose = 0)
            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'batch': batch}


            
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print(best_time)
print(time_hyper)


10/10 - 0s - loss: 0.2118 - accuracy: 0.7331 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2444 - accuracy: 0.7170 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2256 - accuracy: 0.7395 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2691 - accuracy: 0.7235 - 35ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2825 - accuracy: 0.7074 - 37ms/epoch - 4ms/step
10/10 - 0s - loss: 0.1927 - accuracy: 0.7299 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1999 - accuracy: 0.7170 - 31ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2230 - accuracy: 0.7653 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2622 - accuracy: 0.7492 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2916 - accuracy: 0.7460 - 30ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1790 - accuracy: 0.7621 - 33ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1923 - accuracy: 0.7395 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.1968 - accuracy: 0.7460 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2071 - accuracy: 0.7621 - 32ms/epoch - 3ms/step
10/10 - 0s - loss: 0.2549 - accura